In [89]:
import pandas as pd
import numpy as np
from datetime import *

# Restaurant Operating hours data

In [90]:
restaurant_rating_df = pd.read_csv('data/full_restaurant_rating_data.csv')
df_operating_hours = restaurant_rating_df[["mon_operating_hours", "tue_operating_hours", "wed_operating_hours","thu_operating_hours", "fri_operating_hours", "sat_operating_hours", "sun_operating_hours"]].copy()
df_operating_hours

,mon_operating_hours,tue_operating_hours,wed_operating_hours,thu_operating_hours,fri_operating_hours,sat_operating_hours,sun_operating_hours
0,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM
1,NaN,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,11:00 AM - 3:00 PM,11:00 AM - 4:30 PM
2,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM
3,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM
4,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:00 AM - 2:30 PM,11:00 AM - 2:30 PM
...,...,...,...,...,...,...,...
11156,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11157,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11158,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11159,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Total operating hours in a week

In [91]:
df_operating_hours["Total operating hours"] = 0

for i in range (7):
    #Taking daily operating hours out as a dataframe
    df_avg_operating_hours = df_operating_hours.iloc[:, i]

    #Spliting Opening time and Closing time into two columns 
    df_avg_operating_hours = df_avg_operating_hours.str.split("-", expand = True)

    #Purpose of this code is to avoid OutOfBoundsDateTime Errors.
    df_avg_operating_hours[0] =  "2022-03-05 " + df_avg_operating_hours[0] 

    # Special case: When closing hour starts with 12
    # If the opening hour is PM, and closing hour is AM
    # If the opening hour is AM, and closing hour is AM ?? closing - open <= 0, special case: when opening is 12:00 - 12:59 AM, then closing - open >= 0
    # If opening hour is PM, and closing hour is second day PM?? closing - open <= 0, special case: when closing hour is 12:00 - 12:59 PM, then closing - open >= 0

    # when closing is 12:00 - 12:59 AM, set all others to "2022-03-05" first.
    df_avg_operating_hours[1] = np.where(df_avg_operating_hours[1].str.split().str[-1] == "AM", 
                                    np.where(df_avg_operating_hours[1].str.startswith(" 12"), "2022-03-06 " + df_avg_operating_hours[1], "2022-03-05 " + df_avg_operating_hours[1])
                                , "2022-03-05 " + df_avg_operating_hours[1])

    # If the opening hour is PM, and closing hour is AM
    df_avg_operating_hours[1] = np.where(df_avg_operating_hours[1].str.split().str[-1] == "AM", 
                                    np.where(df_avg_operating_hours[0].str.split().str[-1] == "PM", "2022-03-06 " + df_avg_operating_hours[1].str.split().str[1] + " " + df_avg_operating_hours[1].str.split().str[2], df_avg_operating_hours[1]) 
                                    , df_avg_operating_hours[1])

    # If the opening hour is AM, and closing hour is AM and closing - open <= 0.  
    df_avg_operating_hours[1] = np.where(df_avg_operating_hours[1].str.split().str[-1] == "AM", 
                                    np.where(df_avg_operating_hours[0].str.split().str[-1] == "AM", 
                                        np.where((df_avg_operating_hours[1].str.split().str[1].str.replace(":", ".").astype(float) - df_avg_operating_hours[0].str.split().str[1].str.replace(":", ".").astype(float)) <= 0, "2022-03-06 " + df_avg_operating_hours[1].str.split().str[1] + " " + df_avg_operating_hours[1].str.split().str[2] 
                                        ,df_avg_operating_hours[1])
                                    , df_avg_operating_hours[1])
                                ,df_avg_operating_hours[1])
    # Special case: when opening is 12:00 - 12:59 AM, then closing - open >= 0.
    df_avg_operating_hours[1] = np.where(df_avg_operating_hours[1].str.split().str[-1] == "AM", 
                                    np.where(df_avg_operating_hours[0].str.split().str[-1] == "AM", 
                                        np.where(df_avg_operating_hours[0].str.split().str[1].str.startswith("12"), 
                                            np.where((df_avg_operating_hours[1].str.split().str[1].str.replace(":", ".").astype(float) - df_avg_operating_hours[0].str.split().str[1].str.replace(":", ".").astype(float)) <= 0, "2022-03-05 " + df_avg_operating_hours[1].str.split().str[1] + " " + df_avg_operating_hours[1].str.split().str[2] 
                                            ,df_avg_operating_hours[1])
                                        ,df_avg_operating_hours[1])
                                    ,df_avg_operating_hours[1])
                                ,df_avg_operating_hours[1])

    # If the opening hour is PM & not 12:00 - 12:59 PM, and closing hour is PM and closing - open <= 0. 
    df_avg_operating_hours[1] = np.where(df_avg_operating_hours[1].str.split().str[2] == "PM", 
                                    np.where(df_avg_operating_hours[0].str.split().str[2] == "PM", 
                                        np.where(df_avg_operating_hours[0].str.split().str[1].str.startswith("12") == False, 
                                            np.where((df_avg_operating_hours[1].str.split().str[1].str.replace(":", ".").astype(float) - df_avg_operating_hours[0].str.split().str[1].str.replace(":", ".").astype(float)) <= 0, "2022-03-06 " + df_avg_operating_hours[1].str.split().str[1] + " " + df_avg_operating_hours[1].str.split().str[2] 
                                            ,df_avg_operating_hours[1])
                                        , df_avg_operating_hours[1])
                                    ,df_avg_operating_hours[1])
                                ,df_avg_operating_hours[1])

    # special case: when closing hour is 12:00 - 12:59 PM, then closing - open >= 0
    df_avg_operating_hours[1] = np.where(df_avg_operating_hours[1].str.split().str[-1] == "PM", 
                                    np.where(df_avg_operating_hours[0].str.split().str[-1] == "PM", 
                                        np.where(df_avg_operating_hours[1].str.split().str[1].str.startswith("12"), 
                                            np.where((df_avg_operating_hours[1].str.split().str[1].str.replace(":", ".").astype(float) - df_avg_operating_hours[0].str.split().str[1].str.replace(":", ".").astype(float)) >= 0, "2022-03-06 " + df_avg_operating_hours[1].str.split().str[1] + " " + df_avg_operating_hours[1].str.split().str[2] 
                                            ,df_avg_operating_hours[1])
                                        ,df_avg_operating_hours[1])
                                    ,df_avg_operating_hours[1])
                                ,df_avg_operating_hours[1])

    #convert object to datetime for calculation           #.dt.strftime('%H:%M') #show only hours and min but change the data type to string
    df_avg_operating_hours[0] = pd.to_datetime(df_avg_operating_hours[0])
    df_avg_operating_hours[1] = pd.to_datetime(df_avg_operating_hours[1])

    #replacing null values as 0 for calculating total operating hours because null values will affect the sum
    operating_hours = np.where(df_avg_operating_hours[0].isna(), 0, (df_avg_operating_hours[1] - df_avg_operating_hours[0]) / np.timedelta64(1, 'h'))

    df_operating_hours["Total operating hours"] = df_operating_hours["Total operating hours"] + operating_hours 

df_operating_hours


,mon_operating_hours,tue_operating_hours,wed_operating_hours,thu_operating_hours,fri_operating_hours,sat_operating_hours,sun_operating_hours,Total operating hours
0,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,77.0
1,NaN,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,11:00 AM - 3:00 PM,11:00 AM - 4:30 PM,19.5
2,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,63.0
3,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,84.0
4,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:00 AM - 2:30 PM,11:00 AM - 2:30 PM,22.0
...,...,...,...,...,...,...,...,...
11156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
11157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
11158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
11159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


# Number of days operating (Including null values as 0)

In [92]:
#number of days operating per week
for i in range (7):
    if (i == 0):
        df_operating_hours["# of days open"] = np.where(df_operating_hours.iloc[:, i].isna(), 0, 1)
    else:
        df_operating_hours["# of days open"] = np.where(df_operating_hours.iloc[:, i].isna(), df_operating_hours["# of days open"], df_operating_hours["# of days open"] + 1)
df_operating_hours

,mon_operating_hours,tue_operating_hours,wed_operating_hours,thu_operating_hours,fri_operating_hours,sat_operating_hours,sun_operating_hours,Total operating hours,# of days open
0,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,77.0,7
1,NaN,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,11:00 AM - 3:00 PM,11:00 AM - 4:30 PM,19.5,6
2,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,63.0,7
3,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,84.0,7
4,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:00 AM - 2:30 PM,11:00 AM - 2:30 PM,22.0,7
...,...,...,...,...,...,...,...,...,...
11156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
11157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
11158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
11159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0


# Average daily operating hours

In [93]:
df_operating_hours["Average daily operating hours"] = round(df_operating_hours["Total operating hours"] / df_operating_hours["# of days open"], 2) #divide by the days they operates
df_operating_hours

,mon_operating_hours,tue_operating_hours,wed_operating_hours,thu_operating_hours,fri_operating_hours,sat_operating_hours,sun_operating_hours,Total operating hours,# of days open,Average daily operating hours
0,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,77.0,7,11.00
1,NaN,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,11:00 AM - 3:00 PM,11:00 AM - 4:30 PM,19.5,6,3.25
2,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,63.0,7,9.00
3,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,84.0,7,12.00
4,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:00 AM - 2:30 PM,11:00 AM - 2:30 PM,22.0,7,3.14
...,...,...,...,...,...,...,...,...,...,...
11156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN
11157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN
11158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN
11159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN


# Does the restaurant operates on weekends? (Treating null values as False)

In [94]:
# df_operating_hours["Operate_on_only_sat"] = np.where(df_operating_hours["sat_operating_hours"].notna(), np.where(df_operating_hours["sun_operating_hours"].isna(), True, False) , False)
# df_operating_hours["Operate_on_only_sun"] = np.where(df_operating_hours["sun_operating_hours"].notna(), np.where(df_operating_hours["sat_operating_hours"].isna(), True, False) , False)
df_operating_hours["Operate_on_weekends"] = np.where(df_operating_hours["sat_operating_hours"].notna(), np.where(df_operating_hours["sun_operating_hours"].notna(), True, False) , False)
df_operating_hours["Operate_on_satORsun"] = np.where(df_operating_hours["sat_operating_hours"].notna(), True , False)
df_operating_hours["Operate_on_satORsun"] = np.where(df_operating_hours["sun_operating_hours"].notna(), True , df_operating_hours["Operate_on_satORsun"])
df_operating_hours

,mon_operating_hours,tue_operating_hours,wed_operating_hours,thu_operating_hours,fri_operating_hours,sat_operating_hours,sun_operating_hours,Total operating hours,# of days open,Average daily operating hours,Operate_on_weekends,Operate_on_satORsun
0,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,77.0,7,11.00,True,True
1,NaN,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,12:00 PM - 2:30 PM,11:00 AM - 3:00 PM,11:00 AM - 4:30 PM,19.5,6,3.25,True,True
2,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,09:00 AM - 6:00 PM,63.0,7,9.00,True,True
3,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,10:30 AM - 10:30 PM,84.0,7,12.00,True,True
4,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:30 AM - 2:30 PM,11:00 AM - 2:30 PM,11:00 AM - 2:30 PM,22.0,7,3.14,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
11156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,False,False
11157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,False,False
11158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,False,False
11159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,False,False
